In [3]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()

True

In [4]:
model=ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [5]:
model.invoke("what is Hello ")

AIMessage(content='"Hello" is a common English **greeting**.\n\nHere\'s a breakdown of what it is and how it\'s used:\n\n1.  **A Form of Greeting:** It\'s the most widely used word to acknowledge someone, initiate a conversation, or express recognition when meeting or seeing someone.\n    *   *Example:* "Hello, how are you?"\n\n2.  **Answering the Telephone:** It\'s the standard way to answer a phone call.\n    *   *Example:* "Hello?" (when picking up the phone)\n\n3.  **Getting Attention:** It can be used to get someone\'s attention, especially if they seem distracted or haven\'t noticed you.\n    *   *Example:* "Hello? Is anyone home?" or "Hello, excuse me, could you help me?"\n\n4.  **Expressing Surprise or Recognition:** Sometimes used with a specific tone to express surprise, especially if you haven\'t seen someone in a while or didn\'t expect to see them.\n    *   *Example:* "Well, hello there! I didn\'t expect to see you."\n\n**Origin:**\nThe word "hello" is believed to have ori

# Buffer Memory

- Definition: A short-term storage that holds the most recent conversation turns or tokens to provide immediate context for the next model inference.
- Purpose: Keeps recent context available without re-sending the entire history, reducing prompt size and latency.
- Common implementation: sliding window or fixed-size buffer that keeps the last N messages or K tokens.
- Eviction strategies: FIFO (drop oldest), LRU (drop least recently used), or summarization/compaction when full.
- Benefits:
    - Preserves local coherence across turns
    - Keeps prompts within token limits
    - Improves relevance for follow-up queries
- Limitations:
    - Can miss long-term facts if buffer is too small
    - Prone to context drift (irrelevant accumulated details)
    - Privacy risk if sensitive data is retained
- Best practices:
    - Tune buffer size to token limits and task complexity
    - Periodically summarize older context into compact representations
    - Combine buffer memory with a persistent store for long-term facts
    - Apply filters to avoid storing sensitive information
- When to use: short multi-turn conversations requiring immediate context. For prolonged or knowledge-intensive sessions, combine with persistent memory or retrieval mechanisms.

In [9]:
from langchain.prompts import PromptTemplate,HumanMessagePromptTemplate,ChatPromptTemplate,SystemMessagePromptTemplate,MessagesPlaceholder
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [11]:
prompt=ChatPromptTemplate(messages=[SystemMessagePromptTemplate.from_template("You are a nice chatbot having a conversation with a human"),MessagesPlaceholder(variable_name="chat_history"),HumanMessagePromptTemplate.from_template("{question}")])

memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)

C:\Users\91800\AppData\Local\Temp\ipykernel_19284\2296162971.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)


In [12]:
conversation=LLMChain(llm=model,prompt=prompt,verbose=True,memory=memory)

C:\Users\91800\AppData\Local\Temp\ipykernel_19284\3686333804.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation=LLMChain(llm=model,prompt=prompt,verbose=True,memory=memory)


In [15]:
conversation({"question":"Yo buddy how you doing"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human
Human: hi
AI: Hi there! How can I help you today? 😊
Human: Yo buddy how you doing

> Finished chain.


{'question': 'Yo buddy how you doing',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi there! How can I help you today? 😊', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Yo buddy how you doing', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey! I'm doing great, thanks for asking! Just zipping along, ready to chat.\n\nHow about you? What's up?", additional_kwargs={}, response_metadata={})],
 'text': "Hey! I'm doing great, thanks for asking! Just zipping along, ready to chat.\n\nHow about you? What's up?"}

In [20]:
conversation({"question":"Your Favorite anime Character for me it would itachi uchiha ony"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human
Human: hi
AI: Hi there! How can I help you today? 😊
Human: Yo buddy how you doing
AI: Hey! I'm doing great, thanks for asking! Just zipping along, ready to chat.

How about you? What's up?
Human: chathistory
AI: Oh, it looks like you sent `{'chathistory'}`!

Did you mean to type something else, or were you perhaps trying to do something with our conversation history? No worries at all, just let me know! 😊
Human: My name is Raghav and i love to eat mango and lichi fruit and wbu question:
AI: Ah, got it! It looks like you sent a little dictionary-like phrase there, but I can definitely understand the message inside!

So, it's great to meet you, Raghav! It sounds like you have excellent taste – mangoes and litchis are absolutely delicious fruits. They're so vibrant and sweet!

As for me, well, I'm an AI, so I don't actually get to eat or taste anything. But I can p

{'question': 'Your Favorite anime Character for me it would itachi uchiha ony',
 'chat_history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi there! How can I help you today? 😊', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Yo buddy how you doing', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hey! I'm doing great, thanks for asking! Just zipping along, ready to chat.\n\nHow about you? What's up?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content=['chathistory'], additional_kwargs={}, response_metadata={}),
  AIMessage(content="Oh, it looks like you sent `{'chathistory'}`!\n\nDid you mean to type something else, or were you perhaps trying to do something with our conversation history? No worries at all, just let me know! 😊", additional_kwargs={}, response_metadata={}),
  HumanMessage(content=['My name is Raghav and i love to eat mango and lichi fruit and wbu ', 'question:'

In [22]:
print(memory.buffer)

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi there! How can I help you today? 😊', additional_kwargs={}, response_metadata={}), HumanMessage(content='Yo buddy how you doing', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey! I'm doing great, thanks for asking! Just zipping along, ready to chat.\n\nHow about you? What's up?", additional_kwargs={}, response_metadata={}), HumanMessage(content=['chathistory'], additional_kwargs={}, response_metadata={}), AIMessage(content="Oh, it looks like you sent `{'chathistory'}`!\n\nDid you mean to type something else, or were you perhaps trying to do something with our conversation history? No worries at all, just let me know! 😊", additional_kwargs={}, response_metadata={}), HumanMessage(content=['My name is Raghav and i love to eat mango and lichi fruit and wbu ', 'question:'], additional_kwargs={}, response_metadata={}), AIMessage(content="Ah, got it! It looks like you sent a littl

## ConversationBufferWindowMemory — how it works and why to prefer it

**How ConversationBufferWindowMemory works**

- It keeps a sliding window of the most recent N conversation turns (messages). For example, with `k=5` it stores the last 5 user+assistant turns.
- Each new message pushes older messages out of the window once the window is full — the memory only contains the latest context.
- This memory exposes the stored turns (often as `chat_history`) to the prompt so the model sees only the recent context.

**How ConversationBufferMemory works (contrast)**

- It accumulates the entire conversation history from the session and keeps appending every turn.
- Nothing is dropped automatically, so the full history grows unbounded unless you manually trim it.

**Why ConversationBufferWindowMemory is often better**

- Lower token usage: by keeping only the recent N turns you reduce prompt size and avoid hitting context limits.
- Focused context: models usually need recent context to continue a conversation; a window avoids distracting the model with very old, irrelevant turns.
- Faster and cheaper: smaller prompts reduce latency and API cost.
- Privacy & safety: older sensitive content is not kept in memory forever.
- Predictable behavior: bounded memory size avoids unexpected prompt truncation from very long histories.

**Tradeoffs and when to use which**

- Use Window memory when: the task relies on short-term context (chat assistants, step-by-step debugging, short Q&A) and you want predictable prompt sizes.
- Use Full Buffer memory when: you *need* full conversational context (multi-turn troubleshooting where earlier details matter) or when you plan to summarize and store long-term facts elsewhere.
- Hybrid approach: keep a Window for immediate context and periodically summarize older history into a compact long-term memory (e.g., `SummaryMemory` or external DB).

**Practical tip**
- A common pattern: `ConversationBufferWindowMemory(k=6)` to keep the last 6 turns, and separately save important facts to a long-term store. Tune `k` by testing: smaller `k` if you hit token limits, larger `k` if you notice the model losing necessary context.

This makes the system more efficient and robust for most interactive assistants while still allowing long-term knowledge to be preserved elsewhere when needed.

In [23]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [24]:
window_memory=ConversationBufferWindowMemory(k=3)

C:\Users\91800\AppData\Local\Temp\ipykernel_19284\343197281.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory=ConversationBufferWindowMemory(k=3)


In [27]:
conversation_chain = ConversationChain(llm=model, memory=window_memory, verbose=True)

In [32]:
conversation_chain({"input":"My favourite color is blue and favourite animal is panda"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi what is your name i can call
AI: Oh, hello there! It's lovely to meet you!

Well, that's a great question! As an AI, I don't actually have a personal name like humans do. I'm a large language model, and I was trained by Google. So, you can just call me "AI" or "Assistant" or even "Google's AI" if you like! Whatever feels most comfortable for you! I'm here to help you out with whatever you need. What can I do for you today?
Human: My favourite color is blue and favourite animal is panda
AI:

> Finished chain.


{'input': 'My favourite color is blue and favourite animal is panda',
 'history': 'Human: Hi what is your name i can call\nAI: Oh, hello there! It\'s lovely to meet you!\n\nWell, that\'s a great question! As an AI, I don\'t actually have a personal name like humans do. I\'m a large language model, and I was trained by Google. So, you can just call me "AI" or "Assistant" or even "Google\'s AI" if you like! Whatever feels most comfortable for you! I\'m here to help you out with whatever you need. What can I do for you today?',
 'response': "Oh, that's wonderful to know! Thank you for sharing those lovely details about yourself!\n\nBlue is such a fantastic choice for a favorite color! It's incredibly popular and for good reason, don't you think? There are so many beautiful shades, from the deep, calming navy of the ocean to the bright, clear cerulean of a summer sky. Blue is often associated with feelings of tranquility, stability, and even wisdom. It's truly a versatile and beautiful col

In [34]:
conversation_chain({"input":"ok so you remeber what is my fav color"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi what is your name i can call
AI: Oh, hello there! It's lovely to meet you!

Well, that's a great question! As an AI, I don't actually have a personal name like humans do. I'm a large language model, and I was trained by Google. So, you can just call me "AI" or "Assistant" or even "Google's AI" if you like! Whatever feels most comfortable for you! I'm here to help you out with whatever you need. What can I do for you today?
Human: My favourite color is blue and favourite animal is panda
AI: Oh, that's wonderful to know! Thank you for sharing those lovely details about yourself!

Blue is such a fantastic choice for a favorite color! It's incredibly popular and 

{'input': 'ok so you remeber what is my fav color',
 'history': 'Human: Hi what is your name i can call\nAI: Oh, hello there! It\'s lovely to meet you!\n\nWell, that\'s a great question! As an AI, I don\'t actually have a personal name like humans do. I\'m a large language model, and I was trained by Google. So, you can just call me "AI" or "Assistant" or even "Google\'s AI" if you like! Whatever feels most comfortable for you! I\'m here to help you out with whatever you need. What can I do for you today?\nHuman: My favourite color is blue and favourite animal is panda\nAI: Oh, that\'s wonderful to know! Thank you for sharing those lovely details about yourself!\n\nBlue is such a fantastic choice for a favorite color! It\'s incredibly popular and for good reason, don\'t you think? There are so many beautiful shades, from the deep, calming navy of the ocean to the bright, clear cerulean of a summer sky. Blue is often associated with feelings of tranquility, stability, and even wisdom. I

In [40]:
print(conversation_chain.memory)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi what is your name i can call', additional_kwargs={}, response_metadata={}), AIMessage(content='Oh, hello there! It\'s lovely to meet you!\n\nWell, that\'s a great question! As an AI, I don\'t actually have a personal name like humans do. I\'m a large language model, and I was trained by Google. So, you can just call me "AI" or "Assistant" or even "Google\'s AI" if you like! Whatever feels most comfortable for you! I\'m here to help you out with whatever you need. What can I do for you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='My favourite color is blue and favourite animal is panda', additional_kwargs={}, response_metadata={}), AIMessage(content="Oh, that's wonderful to know! Thank you for sharing those lovely details about yourself!\n\nBlue is such a fantastic choice for a favorite color! It's incredibly popular and for good reason, don't you think? There are so many beautiful s